In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from rtmag.process.paper.analytical_field import get_analytic_b_field

from rtmag.process.paper.load import MyModel
device = "cpu"
import rtmag.process.paper.metric as m
from rtmag.process.paper.potential_field import get_potential_field
from rtmag.process.paper.opt import relax
from rtmag.process.paper.opth import draw

In [ ]:
l = 0.3
psi = 0.25*np.pi
B = get_analytic_b_field(m=1, l=l, psi=psi)

In [ ]:
np.save('lowlou_analytic_b_field.npy', B)

In [ ]:
draw(B)

# Potential

In [ ]:
B = np.load('lowlou_analytic_b_field.npy')
B.shape

In [ ]:
bottom = B[:, :, 0, :]
print(bottom.shape)

nx, ny, nz, _ = B.shape
print(nx, ny, nz)

In [ ]:
bp = get_potential_field(bottom[..., 2], nz, l=0.5)

In [ ]:
np.save("lowlou_pot.npy", bp)

In [ ]:
draw(bp)

In [ ]:
res = m.evaluate(bp, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# Optimization

In [ ]:
B = np.load('lowlou_analytic_b_field.npy')
bottom = B[:, :, 0, :]
print(bottom.shape)

In [ ]:
bp = np.load("lowlou_pot.npy")
bp.shape

In [ ]:
bp[:, :, 0, :] = bottom

In [ ]:
br = relax(bp, 'lowlou_opt')

In [ ]:
np.save("lowlou_opt.npy", br)

In [ ]:
draw(br)

In [ ]:
res = m.evaluate(br, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# AI

In [ ]:
meta_path = "D:/models/lowlou/best_model.pt"

mm = MyModel(meta_path, device=device)

In [ ]:
B = np.load('lowlou_analytic_b_field.npy')
B.shape

In [ ]:
b_bottom = B[:, :, 0, :]
b_bottom = b_bottom.transpose((1, 0, 2))[None, None, ...]
print(b_bottom.shape)

In [ ]:
b = mm.get_pred_from_numpy(b_bottom)

In [ ]:
np.save("lowlou_pino.npy", b)

In [ ]:
draw(b)

In [ ]:
res = m.evaluate(b, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# AI + Optimization

In [ ]:
B = np.load('lowlou_analytic_b_field.npy')
b = np.load("lowlou_pino.npy")

In [ ]:
bottom = B[:, :, 0, :]
print(bottom.shape)

In [ ]:
b[:, :, 0, :] = bottom

In [ ]:
br_pino = relax(b, 'lowlou_pino_opt')

In [ ]:
np.save("lowlou_pino_opt.npy", br_pino)

In [ ]:
draw(br_pino)

In [ ]:
res = m.evaluate(br_pino, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")